In [1]:
!pip install --upgrade certifi

In [2]:
import urllib.request
import json
import ssl

In [3]:
import urllib
context = ssl._create_unverified_context()
from urllib.request import Request, urlopen
import json
from io import StringIO

In [4]:
import pandas as pd

In [5]:
from dmtools_client_package.dmtools_client_module import DMToolsClient

In [6]:
import certifi

In [7]:
urllib.request.__version__

'3.12'

In [8]:
## get api key and user id from local .env file
from dotenv import load_dotenv
import os
from os import environ, path
#BASE_DIR = path.abspath(path.dirname(__file__))
BASE_DIR = os.getcwd()
#load_dotenv(path.join(BASE_DIR, ".env"))
load_dotenv(path.join(BASE_DIR, ".env"), override=True)
MY_DMTOOLS_APIKEY = environ.get("MY_DMTOOLS_APIKEY")
MY_DMTOOLS_USERID = environ.get("MY_DMTOOLS_USERID")
Client = DMToolsClient(MY_DMTOOLS_USERID, MY_DMTOOLS_APIKEY)

In [9]:
#BASE_DIR

#MY_DMTOOLS_APIKEY

In [10]:
api_server = "https://dmtools.brown.edu/"
data_api = "dmtool/fastapi_data/open/data/"
fastapi_url_data = api_server + data_api + "update_a_data/?id_in=" + str(1)

In [11]:
request_header = {'dmtool-userid': MY_DMTOOLS_USERID,
 'dmtool-apikey': MY_DMTOOLS_APIKEY,
 'Content-Type': 'application/x-www-form-urlencoded'}
dmtools_userid = MY_DMTOOLS_USERID
dmtools_apikey = MY_DMTOOLS_APIKEY

In [12]:
purpose = 'read_a'
subject = 'data'
id = 1
Client.create_current_url(purpose, subject, id)
a_data_json_obj = Client.read_current_as_json()

In [13]:
a_data_json_obj['data_values']

'{[60 4e-43;70 4e-43;160 5.5e-44;320 2.5e-44;630 3.2e-44;640 3e-44;640 1.2e-44;630 1e-44;225 9e-45;215 7e-45;315 7e-46;470 2.1e-46;650 7e-47;660 6.8e-47;660 2e-47;650 2e-47;470 6e-47;315 1e-46;215 3e-46;100 8e-46;60 1e-45;60 4e-43]}'

In [14]:

context = ssl._create_unverified_context()

# The nested JSON data as a Python list (will be converted to string)
nested_data = [[['1', '1'], ['1', '0']], [['1', '1'], ['1', '0']]]
simple_data = "data"
# Convert the nested data to a JSON string
nested_data_string = json.dumps(nested_data)

# Construct the payload with the nested JSON string
payload = {
    "data": nested_data_string
}

json_data = json.dumps(payload)

payload["data"]

# Convert the payload to a JSON string and encode it to bytes
#json_data = json.dumps(payload).encode('utf-8')

# Create an SSL context (if needed)


# Create the request
#request = urllib.request.Request(fastapi_url_data, data=json_data, headers=request_header, )

#request = urllib.request.Request(fastapi_url_data, headers=request_header, method='PATCH')
#r = urllib.request.urlopen(request, context=context)
#string = r.read().decode('utf-8')



'[[["1", "1"], ["1", "0"]], [["1", "1"], ["1", "0"]]]'

In [27]:
class TestClass():
    def __init__(self, dmtool_userid_in, dmtool_apikey_in):
        self.context = ssl._create_unverified_context()
        self.dmtool_userid = dmtool_userid_in
        self.dmtool_apikey = dmtool_apikey_in
        self.api_server = "https://dmtools.brown.edu/"
        self.data_api = "dmtool/fastapi_data/open/data/"
        self.current_url = self.api_server + self.data_api
        self.current_df = pd.DataFrame()
        self.create_request_header()
    
    def create_request_header(self):
        self.request_header = {'dmtool-userid': self.dmtool_userid ,'dmtool-apikey': self.dmtool_apikey, 'Content-Type': 'application/x-www-form-urlencoded'}
    
    def update_current(self,data_id_in,data_in,url_in):
        if url_in == '':
            encoded_data = data_in.encode('utf-8')
            update_url = self.api_server + self.data_api + "update_a_data/?id_in=" + str(data_id_in)
            update_request = urllib.request.Request(update_url, data=encoded_data, method='PATCH')
            update_request.add_header('dmtool-userid', str(self.dmtool_userid))
            update_request.add_header('dmtool-apikey', self.dmtool_apikey)
            #update_request.add_header('Content-Type', 'application/x-www-form-urlencoded')
            update_request.add_header('Content-Type', 'application/json')
        else:
            encoded_data = data_in.encode('utf-8')
            update_request = urllib.request.Request(url_in, data=encoded_data, method='PATCH')
            update_request.add_header('dmtool-userid', str(self.dmtool_userid))
            update_request.add_header('dmtool-apikey', self.dmtool_apikey)
            update_request.add_header('Content-Type', 'application/json')
            #create_request.add_header('Content-Type', 'application/x-www-form-urlencoded')
        
        try:
            with urllib.request.urlopen(update_request, context=context) as response:
                response_data = response.read().decode('utf-8')
                response_json_obj = json.loads(response_data)
                return response_json_obj
        except urllib.error.HTTPError as e:
            if e.code == 307:
                redirect_url = e.headers['Location']
                return self.update_current(data_id_in,data_in, redirect_url)
            elif e.code == 422:
                print("HTTP 422 Unprocessable Entity")
                error_response = e.read().decode('utf-8')
                return "Response content:", error_response
            else:
                print(f"Error: {e.code}")
                error_response = e.read().decode('utf-8')
                return "Response content:", error_response
    

    def clean_data_values(self, data_id_in):
        fastapi_url_data = self.api_server + self.data_api + "read_a_data/?id_in=" + str(data_id_in)
        request = Request(fastapi_url_data, headers=self.request_header)
        r = urllib.request.urlopen(request, context=self.context)
        string = r.read().decode('utf-8')
        a_data_json_obj = json.loads(string)
        data_string = a_data_json_obj['data_values']
        #all_plots_json_obj[1]
        #a_data_df = pd.DataFrame(a_data_json_obj, index=[0])
        data_string = data_string.replace("{[", "")
        data_string = data_string.replace("]}", "")
        #print(data_string)
        data_series = data_string.split("]")
        len(data_series)
        lol = []
        for l in range(0,len(data_series)):
            series_lol = []
            series_id = 0
            trace_id = l + 1
            single_set = data_series[l]
            set_list = single_set.split(";")
            for i in set_list:
                ## the following was added due to a different approach to data_string format
                r0 = i.replace(',[','')
                r1 = r0.replace('  ',' ')
                r2 = r1.replace('  ',' ')
                r3 = r2.replace('  ',' ')
                r4 = r3.replace('\r\n','')
                r5 = r4.replace('\t',' ')
                r6 = r5.replace(',',' ')
                r7 = r6.replace('  ',' ')
                r8 = r7.replace('  ',' ')
                r9 = r8.replace('\n',' ')
                r10 = r9.replace("', '"," ")
                r11 = r10.replace("['[",'')
                r12 = r11.replace(']','')
                r13 = r12.replace('[','')
                r14 = r13.replace(',','')
                s = r14.lstrip()
                z = s.split(" ");
                try:
                    raw_y = z[1]
                    raw_x = z[0].replace(",[", "")
                    #print('print split z >>>>', z)
                except:
                    #print(z)
                    raw_y = '0'
                    raw_x = '0'
                
                try:
                    x = float(raw_x)
                    y = float(raw_y)
                    masses =  float(raw_x)
                    cross_sections = float(raw_y)
                    formatted_x = "{:.5e}".format(x)
                    formatted_y = "{:.5e}".format(y)
                    #append_this = str(trace_id) + "," + formatted_x + "," + formatted_y
                    #append_this = '['+formatted_x+','+formatted_y+'],'
                    append_this = [formatted_x, formatted_y]
                    series_lol = series_lol + [append_this]
                except:
                    print('rejected z >> ', z)
            lol = lol + [series_lol]
            

        # Convert the nested data to a JSON string

        nested_data_string = json.dumps(lol)
        
        # Construct the payload with the nested JSON string
        payload = {
            "data": nested_data_string
        }
        
        json_data = json.dumps(payload)
        print(json_data)
        r = self.update_current(data_id_in, json_data,'')
        #print(r)

In [28]:
purpose = 'read_all'
subject = 'data'
Client.create_current_url(purpose, subject, id)
all_data_json_obj = Client.read_current_as_json()

In [29]:
#update_current(json_data, fastapi_url_data)
Client_2 = TestClass(MY_DMTOOLS_USERID, MY_DMTOOLS_APIKEY)


In [30]:
all_data_json_obj[1]['id']

'5'

In [31]:
for dic in all_data_json_obj:
    data_id = dic['id']
    rr = Client_2.clean_data_values(data_id)
    print(data_id)

{"data": "[[[\"6.00000e+01\", \"4.00000e-43\"], [\"7.00000e+01\", \"4.00000e-43\"], [\"1.60000e+02\", \"5.50000e-44\"], [\"3.20000e+02\", \"2.50000e-44\"], [\"6.30000e+02\", \"3.20000e-44\"], [\"6.40000e+02\", \"3.00000e-44\"], [\"6.40000e+02\", \"1.20000e-44\"], [\"6.30000e+02\", \"1.00000e-44\"], [\"2.25000e+02\", \"9.00000e-45\"], [\"2.15000e+02\", \"7.00000e-45\"], [\"3.15000e+02\", \"7.00000e-46\"], [\"4.70000e+02\", \"2.10000e-46\"], [\"6.50000e+02\", \"7.00000e-47\"], [\"6.60000e+02\", \"6.80000e-47\"], [\"6.60000e+02\", \"2.00000e-47\"], [\"6.50000e+02\", \"2.00000e-47\"], [\"4.70000e+02\", \"6.00000e-47\"], [\"3.15000e+02\", \"1.00000e-46\"], [\"2.15000e+02\", \"3.00000e-46\"], [\"1.00000e+02\", \"8.00000e-46\"], [\"6.00000e+01\", \"1.00000e-45\"], [\"6.00000e+01\", \"4.00000e-43\"]]]"}
1
{"data": "[[[\"5.00000e+01\", \"5.00000e-43\"], [\"7.50000e+01\", \"7.00000e-43\"], [\"8.00000e+01\", \"4.00000e-42\"], [\"1.50000e+02\", \"1.00000e-41\"], [\"1.50000e+02\", \"3.00000e-42\"],